In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
# pd.set_option('max_columns', 50)

In [3]:
os.getcwd()

'c:\\Users\\admin\\Desktop\\backtest'

In [7]:
df_ret = pd.read_csv(r'c:\Users\admin\Desktop\backtest\ret.csv').set_index('Dates')
df_ret.index = pd.to_datetime(df_ret.index)

In [14]:
df_ret

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-13', '2020-01-14',
               ...
               '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23',
               '2021-12-24', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', name='Dates', length=523, freq=None)

In [ ]:
def pnl_hedge(ser_pnl, df_signal): 
    infp = PurePath('/home/dahmed/data/tree_data/S&P_rolled.parquet') 
    Rm = pd.read_parquet(infp).pct_change() 
    Rs = df_return 6 Rm = Rm.reindex(index = Rs.index).squeeze() 
    
    # calculate rolling beta value 
    num = Rs.mul(Rm, axis=0).rolling(60).mean() 
    den = (Rm**2).rolling(60).mean() 
    betas = num.div(den,0) 
    excess_return = ser_pnl - ( (betas*df_signal.shift(1)).sum(1) * Rm ) 
    
    return excess_return

In [ ]:
def backtest(df_signal, name, beta): 
    df_pnl = df_return * df_signal.shift(1) 
    ser_pnl = df_pnl.sum(1) 
    if beta: ser_pnl = pnl_hedge(ser_pnl, df_signal) 
    cum_pnl = ser_pnl.cumsum() 
    vol = ser_pnl.std() 
    
    # change in the number of the stocks I hold 
    df_number_of_stocks = df_signal/df_price
    ser_trades = (df_number_of_stocks.diff() * df_price).abs().sum(1) 
    ser_gmv = df_signal.abs().sum(1)
    ser_turnover = ser_trades/(ser_gmv.shift(1)) 
    
    # gmv yesterday was zero control 
    counter = 0
    for i in range(len(ser_turnover)): 
        if ser_turnover[i] == np.inf or ser_turnover[i] == -np.inf: counter += 1
    print(f'number of infs: {counter}')
    ser_turnover.replace([np.inf, -np.inf], 0, inplace=True) 
    
    # trades executed (dollar) and turnover
    trades, turn = ser_trades.sum(), ser_turnover.mean() 
    # total gross pnl (dollar) 
    gross_pnl = ser_pnl.sum() 
    # average gross pnl (dollar) 
    avg_gross_pnl = ser_pnl.mean()
    # total net pnl (dollar), 2bps for eu per dollar trade 
    net_pnl = ser_pnl.sum() - trades*2/10000 46 
    # average net pnl for nSR calculation 
    avg_net_pnl = net_pnl/len(ser_pnl) 
    # gross SR 
    # 16 because 256 for mean and 16 for std 
    SR = 16*avg_gross_pnl/vol 
    # net SR 
    nSR = 16*avg_net_pnl/vol 
    # return per dollar traded given in bps 
    gbias = 10000*gross_pnl/trades 
    # net return per dollar traded given in bps 
    nbias = 10000*net_pnl/trades 
    
    # MDD scaled by vol 
    MDD = cum_pnl.cummax().sub(cum_pnl, axis=0).max() 
    MDD = MDD/vol 
    return [cum_pnl.ffill().iloc[-1], SR, nSR, turn*100, 2/turn, gbias, nbias, MDD, vol], cum_pnl/vol, ser_gmv 


In [ ]:
def calculate(*args, beta = False, gmv = False, delta = False, clip = False): 
    df_description = pd.DataFrame(columns=['final gross pnl', 'sharpe ratio', 'net sharpe \n ratio', 'turnover (%)', ' holding period \n (days)', 'gross bias \n (bps)', 'net bias (bps)', 'mdd', 'daily std']) 68 69 idxs = [] 70 pnls = [] 71 market_values = [] 72 73 for i, arg in enumerate(*args): 74 # if arg == 'influential': 75 # df_strat = influential(li) 76 # if arg == 'non influential': 77 # df_strat = influential(lni) 78 if arg == 'hold': 79 df_strat = subset() 80 # if arg == 'baseline': 81 # df_strat = baseline() 82 83 if delta: 84 df_strat = df_strat.sub(df_strat.mean(1), axis = 0) 85 # if gmv: 86 # df_strat = df_strat.div(df_strat.abs().sum(1), axis=0) * 10 87 # if clip: 88 # ser_sum = df_strat.sum(1) 89 90 statistics, cumulative_pnl, mv = backtest(df_strat, arg, beta) 91 pnls.append((cumulative_pnl, arg)) 92 market_values.append(mv) 93 df_description.loc[len(df_description)] = statistics 94 idxs.append(f'{arg}') 95 96 df_description.index = idxs 97 df_description = df_description.round(decimals=4) 98 99 return df_description, pnls, market_values 100 def plots(notes, gmvs, *args, beta = False): 101 102 names = [] 103 104 plt.figure(figsize=(15,7)) 105 for i, arg in enumerate(*args): 106 plt.plot(arg[0].index, arg[0].values, label = f'{arg[1]}') 107 names.append(arg[1]) 108 plt.xlabel('t') 109 110 if beta: 111 plt.title(f'Excess PnL from beta hedging' r' (scaled by $\sigma_{daily}$)' f' | {notes}') 112 plt.ylabel('Excess PnL') 113 else: 8/24/23, 11:16 AM merged.ipynb localhost:44831/c96ec148-732c-414b-bb08-f01a273d52f3/ 3/3 114 plt.title(f'Cummulative PnL' r' (scaled by $\sigma_{daily}$)' f' | {notes}') 115 plt.ylabel('Cummulative PnL') 116 117 plt.legend() 118 plt.grid() 119 120 the_table = plt.table(cellText=full_table.values,colWidths = [0.111]*len(full_table.columns), 121 rowLabels=full_table.index, colLabels=full_table.columns, 122 cellLoc = 'center', rowLoc = 'center', loc='top', bbox=[0, 1.055, 1, 0.35]) 123 124 the_table.auto_set_font_size(False) 125 the_table.set_fontsize(11) 126 plt.show() 127 128 plt.figure(figsize=(5,3)) 129 for i, gmv in enumerate(gmvs): 130 gmv.plot(label = f'{names[i]}') 131 plt.legend() 132 plt.title('gross market value') 133 plt.show() 134 # beta hedge pnl 135 # full_table, signals= calculate((('baseline'),), beta = False) 136 # plots('high market cap stocks', signals, beta = False) 137 # with open('/home/dahmed/pythia/rui_high_stocks.pkl', 'rb') as f: 138 # idqis = pickle.load(f) 139 140 # print(len(idqis)